In [14]:
import torch
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet50_Weights
import numpy as np
import torchvision.transforms as transforms        
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.aps import split_data_set, aps_scores, aps_classification, eval_aps

#  Reprocess
data_transform = transforms.Compose([
    transforms.Resize(232), 
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# load pre-trained model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device)


subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)
model = ModelWithTemperature(model, temperature = 1.8).to(device)
model.set_temperature(train_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Before temperature - NLL: 1.031, ECE: 0.164
Optimal temperature: 0.967
After temperature - NLL: 0.987, ECE: 0.132
APS Classification, Start!

Running experiment 1/10...
q_hat = 0.6867257583150878
Total set size: 1153309
Total coverage sets: 23187
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 46.13236
Average Coverage Rate in runs 1: 92.75%

Running experiment 2/10...
q_hat = 0.6903867567795148
Total set size: 1188733
Total coverage sets: 23258
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 47.54932
Average Coverage Rate in runs 2: 93.03%

Running experiment 3/10...
q_hat = 0.6901133769291092
Total set size: 1193585
Total coverage sets: 23252
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 47.7434
Average Coverage Rate in runs 3: 93.01%

Running experiment 4/10...
q_hat = 0.689825065504718
Total set size: 1200127
Total coverage sets: 23253
Total samples amount: 25000
Average Prediction Set Size Aft

## Result
From the above test, following results can be collected :
- Final Average Prediction Set Size: 46.85 / 1000
- Final Average Coverage: 92.87% ($\alpha$=0.1)